# Facial-Emotion-Recogntion-HCL

## Environment

### System

- Os: Windows 11 64bit
- Python: 3.11.9

### Package

- pytorch: cu126 
- ipykernel: 6.29.5
- pandas: 2.2.2
- Pillow: 10.3.0
- tqdm: 4.66.4
- opencv-python: 4.11.0
- albumentations: 2.06

### Import libraries

In [ ]:
import torch
import gc

import scripts.augment as sa
import scripts.preprocessing as sp
import scripts.evaluate as ev
import scripts.model as md
import scripts.backbone as b

### Define constant

In [ ]:
# === Define paths ===
affectnet_raw_path = "./data/affectnet/" 
processed_root_path = "./data/processed/"  
processed_metadata_csv = "./data/processed/train.csv"  
augmented_root_path = "./data/augmented/"  
augmented_metadata_csv = "./data/augmented/train.csv"  

# === Define preprocessing & augmentation parameters ===
img_size = (224, 224)   
rename_images = True   
dry_run = False       
augment_target_per_class = 8000    
augment_severity = "medium"        

### Data preprocessing

In [ ]:
# Preprocess affectnet data
processed_affectnet = sp.process_affectnet(
    affectnet_root=affectnet_raw_path,
    processed_root=processed_root_path,
    img_size=img_size,
    rename=rename_images,
    rename_width=5,
    dry_run=dry_run
)

### Data augmentation

In [ ]:
# Augment affectnet training split
augmented_affectnet = sa.augment_affectnet(
    processed_root=processed_root_path,
    aug_root=augmented_root_path,
    target=augment_target_per_class,
    severity=augment_severity
)

### Model training and evaluate

In [ ]:
device = torch.device("mps")

model_configs = [
    ("facebook/deit-small-distilled-patch16-224", 5),
    ("facebook/deit-base-distilled-patch16-224", 5),
    ("microsoft/resnet-50", 5),
    ("microsoft/resnet-152", 5),
    ("timm/vit_small_patch16_224.augreg_in21k", 5),
    ("google/vit-base-patch16-224", 5),
]

# Load data once
data_loaders = ev.load_affectnet_data(augmented_metadata_csv)
train_loader = data_loaders['train_loader']
val_loader = data_loaders['val_loader']
test_loader = data_loaders['test_loader']

# Iterate and train each backbone + hybrid classifier
for model_name, epochs in model_configs:
    print(f"\n=== Training pipeline for {model_name} ===")
    # 1. Instantiate and fine-tune backbone
    backbone = b.VisionFeatureExtractor(model_name=model_name, device=device)
    print(f"Fine-tuning {model_name}...")

    backbone.finetune(
        train_loader,
        val_loader,
        num_classes=8,
        epochs=epochs,
        learning_rate=2e-5,
        weight_decay=0.01
    )

    # Clear unused memory to prevent OOM
    del backbone.classification_model
    torch.cuda.empty_cache()
    gc.collect()

    # 2. Train hybrid classifier
    classifier = md.LightGBMClassifier()
    hybrid_model = md.HybridEmotionClassifier(backbone, classifier)
    print(f"Training hybrid model for {model_name}...")
    hybrid_model.train(train_loader, val_loader)

    # 3. Evaluate on test set
    print(f"Evaluating hybrid model for {model_name}...")
    hybrid_model.evaluate(test_loader)

    # 4. Save and cleanup
    save_path = f"models/{model_name.replace('/', '_')}"
    hybrid_model.save(save_path)
    print(f"Saved model to {save_path}")

    # Cleanup GPU memory
    del backbone
    del hybrid_model
    torch.cuda.empty_cache()
    gc.collect()

print("All models trained and saved successfully.")